In [69]:
from smooth import is_smooth, sieve_to, psi_approx, log_dickman, factorize, is_prime
from random import randint, seed, random, Random
from bisect import bisect_left, bisect_right
from collections import namedtuple
import math
import time
import os
import gmpy2

In [42]:
PRIMES = tuple(sieve_to(1 << 22))

In [101]:
def estim_y(p):
    L = math.log(p)
    return int(math.exp(math.sqrt(0.5 * L * math.log(L))))

P = 2**64 - 59
y = estim_y(P)
y

9619

In [102]:

u = math.log(P) / math.log(2 ** 59)
1 / math.exp(log_dickman(u))

1.088548689199712

In [58]:
k = len(sieve_to(y))
k

1187

In [48]:
3.15553e-06 * k * 1800

6.742105398

In [63]:
def find_gen(p):
    primes = [q for (q, _) in factorize(p-1)]

    while True:
        g = randint(2, p - 1)
        valid = True
        for q in primes:
            if pow(g, (p - 1) // q, p) == 1:
                valid = False
                break
        if valid:
            return g

g = find_gen(P)
g

16355709244876859514

In [8]:
def build_product_tree(values, k):
    levels = []
    level = [gmpy2.mpz(p) for p in values[:k]]
    levels.append(level)
    while len(level) > 1:
        next_level = []
        for i in range(0, len(level), 2):
            if i + 1 < len(level):
                next_level.append(level[i] * level[i+1])
            else:
                next_level.append(level[i])
        levels.append(next_level)
        level = next_level
    return levels

def batch_remainder(p_levels, X):
    Z = p_levels[-1][0]

    x_levels = build_product_tree(X, len(X))
    h = len(x_levels) - 1

    root_prod = x_levels[h][0]
    rems = [gmpy2.t_mod(Z, root_prod)]

    for level in range(h, 0, -1):
        parent_rems = rems
        node_products = x_levels[level - 1]
        size = len(node_products)
        rems = [None] * size
        for idx in range(size):
            rems[idx] = gmpy2.t_mod(parent_rems[idx // 2], node_products[idx])
    return rems

def batch_smoothness_test(p_levels, X):
    rems = batch_remainder(p_levels, X)
    
    max_x = max(X)
    M = 1 << (max_x.bit_length())
    
    smooth_cands = set()
    for x_mpz, r in zip(X, rems):
        y = gmpy2.powmod(r, M, x_mpz)
        if gmpy2.gcd(x_mpz, y) == x_mpz:
            smooth_cands.add(x_mpz)
    
    return smooth_cands

In [11]:
levels = build_product_tree(PRIMES, k)

In [9]:
for i in range(18):
    X = [gmpy2.mpz(randint(0, P-2)) for _ in range(int(2 ** i))]
    t1 = time.time()
    x = batch_smoothness_test(levels, X)
    print(i, (time.time() - t1))

0 0.0016660690307617188
1 5.1975250244140625e-05
2 5.1975250244140625e-05
3 6.890296936035156e-05
4 0.00011372566223144531
5 0.000209808349609375
6 0.00034499168395996094
7 0.0009310245513916016
8 0.0016162395477294922
9 0.0032279491424560547
10 0.004707813262939453
11 0.0077669620513916016
12 0.013511180877685547
13 0.022312164306640625
14 0.04027700424194336
15 0.07741594314575195
16 0.16199302673339844
17 0.3375709056854248


In [16]:
X = [gmpy2.mpz(randint(0, P-2)) for _ in range(131072)]
t1 = time.time()
x = batch_smoothness_test(levels, X)
print(time.time() - t1)

0.8504021167755127


In [98]:
(332306998946228905391729763996928922).bit_length()

118

In [46]:
arr = list(x)
arr[0].bit_length()

127

In [48]:
t1 = time.time()
print(extract_prime_power_factors(levels, arr[0]))
print(time.time() - t1)

[(686143, 1), (389083, 1), (213391, 1), (67607, 1), (19081, 1), (13997, 1), (2857, 1), (857, 1), (7, 1), (2, 3)]
0.0037851333618164062


In [21]:
def extract_prime_divisors(p_levels, d):
    primes = []
    h = len(p_levels) - 1
    stack = [(h, 0)]
    
    while stack:
        level, idx = stack.pop()
        P = p_levels[level][idx]
        g = gmpy2.gcd(d, P)
        if g == 1:
            continue
        if level == 0:
            primes.append(P)
        else:
            left_idx = 2 * idx
            right_idx = left_idx + 1
            stack.append((level - 1, left_idx))
            if right_idx < len(p_levels[level - 1]):
                stack.append((level - 1, right_idx))
    return primes

def extract_prime_power_factors(p_levels, d):
    factorization = []
    primes = extract_prime_divisors(p_levels, d)
    for p in primes:
        exp = 0
        while d % p == 0:
            d //= p
            exp += 1
        factorization.append((int(p), exp))
    return factorization